In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../.."))
from create_fashion_items_image import create_cw_image
from fashion_class.ImageStruct import ImageStruct

import importlib
from fashion_class.CapsuleWardrobeClass import CapsuleWardrobe
from fashion_class.FashionItem import FashionItem
import gc
import random
from CreateCW import change_item_recommandation, search_alternate_item
from itertools import chain

import torch


In [ ]:



class VersatilityCapsuleWardrobe(CapsuleWardrobe):
    def __init__(self, initial_items, target_items):
        super().__init__(initial_items=initial_items)
        self.NEAREST_ITEM_LENGTH = 100
        self.target_items = target_items

    def calc_versatility_increase(self, items: list[FashionItem], new_item: FashionItem):
        category = new_item.get_category()
        covered_item_ids = set()

        for i in items:
            X = i.expand(self.target_items[category].size())
             # ユークリッド距離を計算します
            distances = torch.sqrt(torch.sum((X - self.target_items[category]) ** 2, dim=1))
            # 上位k個の最小値のインデックスを取得
            sorted_indices = torch.argsort(distances)
            k_min_indices = sorted_indices[:self.NEAREST_ITEM_LENGTH]
            labels: set[int] = set(k_min_indices.tolist())
            covered_item_ids.union(labels)

        pre_score = len(covered_item_ids)
        X = new_item.expand(self.target_items[category].size())
        # ユークリッド距離を計算します
        distances = torch.sqrt(torch.sum((X - self.target_items[category]) ** 2, dim=1))
        # 上位k個の最小値のインデックスを取得
        sorted_indices = torch.argsort(distances)
        k_min_indices = sorted_indices[:self.NEAREST_ITEM_LENGTH]
        labels: list[int] = k_min_indices.tolist()
        covered_item_ids.union(labels)

        return (len(covered_item_ids) - pre_score) / self.NEAREST_ITEM_LENGTH

    def calc_compatibility_increase(self):
        return 0

    def calc_self_cw_compatibility(self):
        return 0

class ProposalCapsuleWardrobe(CapsuleWardrobe):
    def __init__(self, initial_items, target_items):
        super().__init__(initial_items=initial_items)
        self.target_tops = target_items['tops']
        self.target_bottoms = target_items['bottoms']
        self.target_shoes = target_items['shoes']

    def calc_compatibility_increase(self):
        return 0

    def calc_self_cw_compatibility(self):
        return 0


In [ ]:
import copy

def create_cw(cw: CapsuleWardrobe, dataset, eps=1e-3):
    increase = eps + 1
    roop = 0
    pre_cw = copy.deepcopy(cw)
    while increase > eps:
        # ここ遅い。topsとかカテゴリごとに、キャッシュしてあげたほうがいい。
        increase = cw.optimize(dataset)
        pre_cw = copy.deepcopy(cw)
        if increase < 0:
            cw = pre_cw
    return cw

In [ ]:
annotation_file, tensor_file = \
        'D:/M1/fashion/optimization/data/anotation_new.csv', 'D:/M1/fashion/optimization/data/resnet50_tensor.pt'

dataset = ImageStruct(annotation_file, tensor_file, init_item_length=500)
target_items = {
    'tops':torch.stack([i.img_tensor for i in dataset.get_tops()]),
    'bottoms':torch.stack([i.img_tensor for i in dataset.get_bottoms()]),
    'shoes':torch.stack([i.img_tensor for i in dataset.get_shoes()]),
}
create_cw()
